In [1]:
!pip install PyGithub dspy openai

Defaulting to user installation because normal site-packages is not writeable


In [12]:
import openai
import os
import requests
import shutil 
from github import Github
import dspy

In [3]:
github_token = os.getenv('GITHUB_TOKEN')  
openai.api_key = os.getenv('OPENAI_API_KEY')

In [4]:
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
base_directory = os.path.join(desktop_path, "repositories/")

if not os.path.exists(base_directory):
    os.makedirs(base_directory)
    print(f"Created directory: {base_directory}")
else:
    print(f"Directory already exists: {base_directory}")

Directory already exists: C:\Users\jimmy\Desktop\repositories/


In [5]:
def summarize_content(content):
    """Summarize the content of a file using OpenAI API."""
    try:
        max_length = 1000  # AI token limits
        if len(content) > max_length:
            content = content[:max_length]

        response = openai.Completion.create(
            engine="gpt-4.0-mini",
            prompt=f"Summarize the following:\n{content}",
            max_tokens=150
        )
        return response.choices[0].text.strip()
    except Exception as e:
        print(f"Error summarizing content: {e}")
        return None

In [6]:
def access_github_repo(repo_url):
    """Access GitHub repository and download its content"""
    g = Github(github_token)
    repo_name = repo_url.split('/')[-1]

    repo = g.get_repo(repo_url.replace("https://github.com/", ""))

    repo_path = os.path.join(base_directory, repo_name)
    if not os.path.exists(repo_path):
        os.makedirs(repo_path)

    for content_file in repo.get_contents(""):
        file_path = os.path.join(repo_path, content_file.name)
        if content_file.type == "file":
            with open(file_path, "wb") as f:
                f.write(requests.get(content_file.download_url).content)
        elif content_file.type == "dir":
            os.makedirs(file_path)
    return repo_path

In [7]:
def find_missing_files(repo1_path, repo2_path):
    """Find missing files between 2 repos"""
    missing_files = []
    repo1_files = set(os.listdir(repo1_path))
    repo2_files = set(os.listdir(repo2_path))
    for file in repo1_files:
        if file not in repo2_files:
            missing_files.append(file)
    return missing_files

In [8]:
def unpack_and_install(repo_path):
    """unpack and decompress any files and execute the installation"""
    for root, dirs, files in os.walk(repo_path):
        for file in files:
            if file.endswith(".tar.gz") or file.endswith(".zip"):
                shutil.unpack_archive(os.path.join(root, file), extract_dir=root)
                os.system(f"ansible-playbook {os.path.join(root, 'install.yml')}")

In [9]:
def setup_docker_vm():
    """Setup Docker container or vm using dspy"""
    vm = dspy.create_virtual_machine("ubuntu", "my_vm")
    dspy.run_command(vm, "sudo apt update && sudo apt install -y some_package")

    os.system("docker pull ubuntu")
    os.system("docker run -d --name my_container ubuntu")

In [11]:
def run_project(repo_url1, repo_url2):
    """Run the full project"""
    repo1_path = access_github_repo(repo_url1)
    repo2_path = access_github_repo(repo_url2)
    
    with open(os.path.join(repo1_path, "README.md"), "r") as f:
        content = f.read()
        summary = summarize_content(content)
        print("Repository 1 README Summary:", summary)
    
    with open(os.path.join(repo2_path, "README.md"), "r") as f:
        content = f.read()
        summary = summarize_content(content)
        print("Repository 2 README Summary:", summary)

    missing_files = find_missing_files(repo1_path, repo2_path)
    if missing_files:
        print("Missing files:", missing_files)
    else:
        print("No missing files found.")

    unpack_and_install(repo1_path)
    setup_docker_vm()